In [1]:
import hail as hl 
import pandas as pd 

hl.init()

Running on Apache Spark version 3.1.2
SparkUI available at http://192-168-1-12.tpgi.com.au:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.73-c6f6f09cec08
LOGGING: writing to /Users/elston.dsouza/Projects/UTR-Visualisation-App/pipeline/hail-20210817-2159-0.2.73-c6f6f09cec08.log


In [2]:
gnomad = hl.read_table("gs://gcp-public-data--gnomad/release/3.1.1/ht/genomes/gnomad.genomes.v3.1.1.sites.ht")


In [3]:
utr_regions = hl.Table.from_pandas(pd.read_csv("UTR_regions.tsv", sep="\t", names = ["CHROM", "POS", "POS_TO"]))

# Parse intervals
utr_regions = utr_regions.annotate(interval = hl.parse_locus_interval(hl.format("chr%s:%s-%s", 
                                                                                utr_regions.CHROM,
                                                                                utr_regions.POS,
                                                                                utr_regions.POS_TO), 
                                                                     reference_genome='GRCh38'))
utr_regions = utr_regions.key_by("interval")

In [4]:
gnomad = gnomad.filter(hl.is_defined(utr_regions[gnomad.locus]))

In [5]:
vep_input = gnomad.select(gnomad.vep.input)

In [ ]:
vep_input.export("gnomad_v.3.1.1_utr_variants_vep_input.tsv.bgz")

2021-08-17 22:23:09 Hail: INFO: Ordering unsorted dataset with network shuffle


In [4]:
utr_regions.count()
gnomad.show(10)
gnomad.describe()
vep_df

27206